# Imports

In [1]:
import sys, os 
import json
import random
import time
import numpy as np
from numpy import float32
from abc import abstractmethod
from pprint import pprint
import libspn as spn

import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt

from spn_topo.tbm.dataset import TopoMapDataset
from spn_topo.tbm.template import EdgeTemplate, NodeTemplate, PairTemplate, SingletonTemplate, \
    ThreeNodeTemplate, PairEdgeTemplate, SingleEdgeTemplate
from spn_topo.tbm.spn_template import TemplateSpn, NodeTemplateSpn, EdgeTemplateSpn, InstanceSpn
from spn_topo.tests.tbm.runner import TbmExperiment
from spn_topo.tests.runner import TestCase
from spn_topo.util import CategoryManager, print_banner, print_in_box, ColdDatabaseManager

# Training Parameters

In [2]:
results_dir = "/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results"
topo_map_db_root = "/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/data/topo_map"

seed = 100

# Config
train_kwargs = {
    'num_partitions': 10,
    'num_batches': 10,
    'save': True,
    'load_if_exists': True,
    'likelihood_thres': 0.5,
    'will_upsample': TbmExperiment.check_doorway_connecting,
    'upsample_rate': 10,
    'save_training_info': True,

    # spn_structure
    'num_decomps': 1,
    'num_subsets': 2,
    'num_mixtures': 2,
    'num_input_mixtures': 2
}

test_kwargs = {
    'num_partitions': 1,
    'high_likelihood': (0.5, 0.80),
    'low_likelihood': (0.20, 0.50),
}
spn_params = {k:train_kwargs[k] for k in ['num_decomps', 'num_subsets', 'num_mixtures', 'num_input_mixtures']}

# Load topo map dataset

In [3]:
coldmgr = ColdDatabaseManager("Stockholm", "/home/zkytony/sara/sara_ws/src/sara_processing/sara_cold_processing/forpub/COLD")
dataset = TopoMapDataset("/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/data/topo_map")
dataset.load("Small")
topo_maps = dataset.get_topo_maps()
seq_id = list(topo_maps.keys())[0]
print(seq_id)
topo_map = topo_maps[seq_id]
print(len(topo_map.nodes))

floor4_night_c
16


# Create an SPN for Node Templates

In [59]:
three_node_spn = NodeTemplateSpn(ThreeNodeTemplate, seed=seed, **spn_params)
pair_node_spn = NodeTemplateSpn(PairTemplate, seed=seed, **spn_params)
single_node_spn = NodeTemplateSpn(SingletonTemplate, seed=seed, **spn_params)

Generating SPN structure...
[Using seed 100]
Generating weight initialization Ops...
Initializing learning Ops...
Generating SPN structure...
[Using seed 100]
Generating weight initialization Ops...
Initializing learning Ops...
Generating SPN structure...
[Using seed 100]
Generating weight initialization Ops...
Initializing learning Ops...


### Create InstanceSPN

In [61]:
spns = [three_node_spn, pair_node_spn, single_node_spn]
exp = TbmExperiment(topo_map_db_root, *spns,
                    root_dir=results_dir, name="B_InstanceSpnExperiment_NodeTemplate")
spn_paths = {model.template.__name__:exp.model_save_path(model) for model in spns}
spns_tmpls = [(spns[i], spns[i].template) for i in range(len(spns))]
print(spn_paths)

{'PairTemplate': '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_NodeTemplate/models/PairTemplate_0.spn', 'SingletonTemplate': '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_NodeTemplate/models/SingletonTemplate_0.spn', 'ThreeNodeTemplate': '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_NodeTemplate/models/ThreeNodeTemplate_0.spn'}


In [62]:
sess = tf.Session()
instance_spn = InstanceSpn(topo_map, sess, *spns_tmpls, num_partitions=test_kwargs['num_partitions'],
                           seq_id = seq_id, spn_paths=spn_paths, divisions = 8)

[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_NodeTemplate/models/ThreeNodeTemplate_0.spn'


Creating spn for partiion 1
Will duplicate template 3 times.


[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_NodeTemplate/models/ThreeNodeTemplate_0.spn'


Duplicating... 1


[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_NodeTemplate/models/ThreeNodeTemplate_0.spn'


...

Duplicating... 2
Will duplicate template 3 times.


[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_NodeTemplate/models/SingletonTemplate_0.spn'


Duplicating... 1


[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_NodeTemplate/models/SingletonTemplate_0.spn'


Duplicating... 2
Duplicating... 3


In [63]:
spn.display_spn_graph(instance_spn._root, skip_params=False)

In [64]:
instance_spn.expand()

Expanding...


In [65]:
spn.display_spn_graph(instance_spn._root, skip_params=False)

In [67]:
instance_spn._template_iv_map[0]['semantic']
sess.close()

IVs_23

# Create an SPN for Edge Templates

In [4]:
pair_edge_spn = EdgeTemplateSpn(PairEdgeTemplate, seed=seed, **spn_params)
single_edge_spn = EdgeTemplateSpn(SingleEdgeTemplate, seed=seed, **spn_params)

Generating SPN structure...
[Using seed 100]
Generating weight initialization Ops...
Initializing learning Ops...
Generating SPN structure...
[Using seed 100]
Generating weight initialization Ops...
Initializing learning Ops...


### Create InstanceSPN

In [5]:
spns = [pair_edge_spn, single_edge_spn]
exp = TbmExperiment(topo_map_db_root, *spns,
                    root_dir=results_dir, name="B_InstanceSpnExperiment_EdgeTemplate")
spn_paths = {model.template.__name__:exp.model_save_path(model) for model in spns}
spns_tmpls = [(spns[i], spns[i].template) for i in range(len(spns))]
print(spn_paths)

{'PairEdgeTemplate': '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_EdgeTemplate/models/PairEdgeTemplate_0.spn', 'SingleEdgeTemplate': '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_EdgeTemplate/models/SingleEdgeTemplate_0.spn'}


In [7]:
sess = tf.Session()
instance_spn = InstanceSpn(topo_map, sess, *spns_tmpls, num_partitions=test_kwargs['num_partitions'],
                           seq_id = seq_id, spn_paths=spn_paths, divisions = 8)

[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_EdgeTemplate/models/PairEdgeTemplate_0.spn'


Creating spn for partiion 1


[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_EdgeTemplate/models/PairEdgeTemplate_0.spn'


Duplicating... 1


[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_EdgeTemplate/models/PairEdgeTemplate_0.spn'
[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_EdgeTemplate/models/PairEdgeTemplate_0.spn'


...

Duplicating... 8


[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_EdgeTemplate/models/PairEdgeTemplate_0.spn'


Duplicating... 9


[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_EdgeTemplate/models/SingleEdgeTemplate_0.spn'
[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/spn_topo/experiments/results/B_InstanceSpnExperiment_EdgeTemplate/models/SingleEdgeTemplate_0.spn'


Duplicating... 10
Duplicating... 1
Duplicating... 2


In [8]:
spn.display_spn_graph(instance_spn._root, skip_params=False)

In [9]:
instance_spn.expand()

Expanding...


In [10]:
spn.display_spn_graph(instance_spn._root, skip_params=False)